In [1]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

class Reader:
    def __init__(self, id):
        self.id = id
        self.weight = 0
        self.cost = 0
        self.avg_weight_neighbors = 0
        self.avg_cost_neighbors = 0
        self.status = 0  # Target output (0 for OFF, 1 for ACTIVE)

    def set_attributes(self, weight, cost, avg_weight_neighbors, avg_cost_neighbors, status):
        self.weight = weight
        self.cost = cost
        self.avg_weight_neighbors = avg_weight_neighbors
        self.avg_cost_neighbors = avg_cost_neighbors
        self.status = status

class MachineLearningStage:
    def __init__(self, num_readers, num_tags, alpha, beta):
        self.num_readers = num_readers
        self.num_tags = num_tags
        self.alpha = alpha
        self.beta = beta
        self.readers = [Reader(i) for i in range(num_readers)]
        self.X = []
        self.y = []

    def run_simulation(self):
        for reader in self.readers:
            reader.weight = np.random.randint(0, self.num_tags)
            reader.cost = reader.weight * self.alpha
            reader.avg_weight_neighbors = np.random.randint(0, self.num_tags)
            reader.avg_cost_neighbors = reader.avg_weight_neighbors * self.alpha
            reader.status = np.random.randint(0, 2)

    def collect_training_data(self):
        self.X = np.array([(reader.weight, reader.cost, reader.avg_weight_neighbors, reader.avg_cost_neighbors) for reader in self.readers])
        self.y = np.array([reader.status for reader in self.readers])

    def balance_training_data(self):
        # Balancing the dataset
        active_indices = np.where(self.y == 1)[0]
        inactive_indices = np.where(self.y == 0)[0]
        num_samples = min(len(active_indices), len(inactive_indices))
        selected_indices = np.concatenate((active_indices[:num_samples], inactive_indices[:num_samples]))
        self.X = self.X[selected_indices]
        self.y = self.y[selected_indices]

    def train_neural_network(self):
        self.nn_model = MLPClassifier(hidden_layer_sizes=(16, 16), activation='relu', solver='adam', max_iter=500, random_state=42)
        self.nn_model.fit(self.X, self.y)

    def broadcast_trained_weights(self):
        weights = [layer_weights for layer_weights in self.nn_model.coefs_]
        return weights

# Simulation parameters
num_readers = 100
num_tags = 1000
alpha = 0.5
beta = 1.25

# Run machine learning stage
ml_stage = MachineLearningStage(num_readers, num_tags, alpha, beta)
ml_stage.run_simulation()
ml_stage.collect_training_data()
ml_stage.balance_training_data()
ml_stage.train_neural_network()
trained_weights = ml_stage.broadcast_trained_weights()

print("Trained weights broadcasted to each reader.")


Trained weights broadcasted to each reader.


In [2]:
import numpy as np
import time

class Reader:
    def __init__(self, id, num_readers, alpha, threshold):
        self.id = id
        self.num_readers = num_readers
        self.alpha = alpha
        self.threshold = threshold
        self.status = "LOCK"
        self.local_vertex = None
        self.neighbors = []
        self.cost = 0
        self.nn_model = None

    def set_local_vertex(self, num_neighbors):
        if num_neighbors <= self.alpha:
            self.local_vertex = {"weight": num_neighbors}

    def add_neighbor(self, neighbor_id):
        self.neighbors.append(neighbor_id)

    def remove_neighbor(self, neighbor_id):
        if neighbor_id in self.neighbors:
            self.neighbors.remove(neighbor_id)

    def compute_cost(self):
        if self.local_vertex:
            self.cost = self.local_vertex["weight"] * self.id

    def prepare_features(self):
        features = [self.id, len(self.neighbors), self.local_vertex["weight"], self.cost]
        return np.array(features)

    def predict_activation(self):
        features = self.prepare_features()
        return self.nn_model.predict([features])[0]

    def prepare_signals(self):
        signals = []
        if self.local_vertex:
            for neighbor_id in self.neighbors:
                signals.append((self.id, neighbor_id, "UPDATE", self.cost))
        return signals

    def send_signals(self, signals, all_readers):
        for signal in signals:
            sender_id, receiver_id, code, value = signal
            receiver = all_readers[receiver_id]
            if receiver.status == "ACTIVE":
                receiver.receive_signal(sender_id, code, value)

    def receive_signal(self, sender_id, code, value):
        if code == "UPDATE":
            self.cost = value
        elif code == "ACTIVATE":
            self.status = "ACTIVE"
        elif code == "DEACTIVATED":
            self.status = "OFF"

    def make_local_decision(self, all_readers):
        if not self.neighbors:  # No neighbors left
            self.status = "OFF"
            for neighbor_id in range(self.num_readers):
                if neighbor_id != self.id:
                    all_readers[neighbor_id].receive_signal(self.id, "DEACTIVATED", "N/A")
            return True

        max_activation = max(self.neighbors, key=lambda x: all_readers[x].predict_activation())
        if max_activation == self.id:
            self.status = "ACTIVE"
            for neighbor_id in self.neighbors:
                all_readers[neighbor_id].receive_signal(self.id, "ACTIVATE", "N/A")
            return True
        return False

class ApplicationStage:
    def __init__(self, num_readers, alpha, threshold, trained_weights):
        self.num_readers = num_readers
        self.alpha = alpha
        self.threshold = threshold
        self.readers = [Reader(i, num_readers, alpha, threshold) for i in range(num_readers)]
        self.set_trained_weights(trained_weights)

    def set_trained_weights(self, trained_weights):
        for reader, weights in zip(self.readers, trained_weights):
            reader.nn_model = MLPClassifier(hidden_layer_sizes=(16, 16), activation='relu', solver='adam', max_iter=1, random_state=42)
            reader.nn_model.coefs_ = weights

    def run_application_stage(self):
        for reader in self.readers:
            for neighbor_id in range(self.num_readers):
                if neighbor_id != reader.id:
                    if np.random.rand() < 0.5:  # Randomly connect neighbors
                        reader.add_neighbor(neighbor_id)
            reader.set_local_vertex(len(reader.neighbors))
            reader.compute_cost()

        all_signals = []
        for reader in self.readers:
            signals = reader.prepare_signals()
            all_signals.extend(signals)

        while True:
            for reader in self.readers:
                if reader.status == "ACTIVE":
                    reader.status = "OPEN"
                    reader.send_signals(all_signals, self.readers)
                    time.sleep(0.1)  # Simulate waiting for signals

            for reader in self.readers:
                if reader.status == "ACTIVE":
                    reader.status = "LOCK"
                    reader.make_local_decision(self.readers)
            
            active_readers = sum(1 for reader in self.readers if reader.status == "ACTIVE")
            if active_readers == 0:
                print("All readers turned off.")
                break
            elif active_readers == 1:
                print(f"Reader {next(reader.id for reader in self.readers if reader.status == 'ACTIVE')} turned active.")
                break

# Application stage parameters
trained_weights = trained_weights  # Assume we already have trained weights
alpha = 0.5
threshold = 10

# Run application stage
app_stage = ApplicationStage(num_readers, alpha, threshold, trained_weights)
app_stage.run_application_stage()


All readers turned off.


In [3]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

class Reader:
    def __init__(self, id):
        self.id = id
        self.weight = 0
        self.cost = 0
        self.avg_weight_neighbors = 0
        self.avg_cost_neighbors = 0
        self.status = 0  # Target output (0 for OFF, 1 for ACTIVE)

    def set_attributes(self, weight, cost, avg_weight_neighbors, avg_cost_neighbors, status):
        self.weight = weight
        self.cost = cost
        self.avg_weight_neighbors = avg_weight_neighbors
        self.avg_cost_neighbors = avg_cost_neighbors
        self.status = status

class MachineLearningStage:
    def __init__(self, num_readers, num_tags, alpha, beta):
        self.num_readers = num_readers
        self.num_tags = num_tags
        self.alpha = alpha
        self.beta = beta
        self.readers = [Reader(i) for i in range(num_readers)]
        self.X = []
        self.y = []

    def run_simulation(self):
        for reader in self.readers:
            reader.weight = np.random.randint(0, self.num_tags)
            reader.cost = reader.weight * self.alpha
            reader.avg_weight_neighbors = np.random.randint(0, self.num_tags)
            reader.avg_cost_neighbors = reader.avg_weight_neighbors * self.alpha
            reader.status = np.random.randint(0, 2)

    def collect_training_data(self):
        self.X = np.array([(reader.weight, reader.cost, reader.avg_weight_neighbors, reader.avg_cost_neighbors) for reader in self.readers])
        self.y = np.array([reader.status for reader in self.readers])

    def balance_training_data(self):
        # Balancing the dataset
        active_indices = np.where(self.y == 1)[0]
        inactive_indices = np.where(self.y == 0)[0]
        num_samples = min(len(active_indices), len(inactive_indices))
        selected_indices = np.concatenate((active_indices[:num_samples], inactive_indices[:num_samples]))
        self.X = self.X[selected_indices]
        self.y = self.y[selected_indices]

    def train_neural_network(self):
        self.nn_model = MLPClassifier(hidden_layer_sizes=(16, 16), activation='relu', solver='adam', max_iter=500, random_state=42)
        self.nn_model.fit(self.X, self.y)

    def broadcast_trained_weights(self):
        weights = [layer_weights for layer_weights in self.nn_model.coefs_]
        return weights

# Simulation parameters
num_readers = 100
num_tags = 1000
alpha = 0.5
beta = 1.25

# Run machine learning stage
ml_stage = MachineLearningStage(num_readers, num_tags, alpha, beta)
ml_stage.run_simulation()
ml_stage.collect_training_data()
ml_stage.balance_training_data()
ml_stage.train_neural_network()
trained_weights = ml_stage.broadcast_trained_weights()

print("Trained weights broadcasted to each reader.")


Trained weights broadcasted to each reader.


In [5]:
import numpy as np
import time

class Reader:
    def __init__(self, id, num_readers, alpha, threshold):
        self.id = id
        self.num_readers = num_readers
        self.alpha = alpha
        self.threshold = threshold
        self.status = "LOCK"
        self.local_vertex = None
        self.neighbors = []
        self.cost = 0
        self.nn_model = None

    def set_local_vertex(self, num_neighbors):
        if num_neighbors <= self.alpha:
            self.local_vertex = {"weight": num_neighbors}

    def add_neighbor(self, neighbor_id):
        self.neighbors.append(neighbor_id)

    def remove_neighbor(self, neighbor_id):
        if neighbor_id in self.neighbors:
            self.neighbors.remove(neighbor_id)

    def compute_cost(self):
        if self.local_vertex:
            self.cost = self.local_vertex["weight"] * self.id

    def prepare_features(self):
        features = [self.id, len(self.neighbors), self.local_vertex["weight"], self.cost]
        return np.array(features)

    def predict_activation(self):
        features = self.prepare_features()
        return self.nn_model.predict([features])[0]

    def prepare_signals(self):
        signals = []
        if self.local_vertex:
            for neighbor_id in self.neighbors:
                signals.append((self.id, neighbor_id, "UPDATE", self.cost))
        return signals

    def send_signals(self, signals, all_readers):
        for signal in signals:
            sender_id, receiver_id, code, value = signal
            receiver = all_readers[receiver_id]
            if receiver.status == "ACTIVE":
                receiver.receive_signal(sender_id, code, value)

    def receive_signal(self, sender_id, code, value):
        if code == "UPDATE":
            self.cost = value
        elif code == "ACTIVATE":
            self.status = "ACTIVE"
